In [ ]:
# Extracts only the other features from brain MRI

import cv2
import os
import sys
import glob
import csv
import imageio
from skimage import color
from skimage.transform import resize
from scipy import ndimage
from scipy.stats import kurtosis, skew
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage.feature import local_binary_pattern, greycomatrix, greycoprops
from skimage.color import label2rgb
from skimage.filters.rank import entropy
from skimage.morphology import disk
import mahotas as mt
import pywt
import pywt.data
from numpy.testing import assert_array_almost_equal

q = "'"

# Image folder path
path = '...'
path_out = '...'
list_img = os.listdir(path) 
# number_files = number of files in the input folder
number_files = len(list_img)
print(number_files)

i = 0
j = 1
# Input image folder is 'yes'
features = [[0 for x in range(5)] for y in range(number_files+1)]
features[0][0] = 'Image'
features[0][1] = 'Entropy'
features[0][2] = 'LBP'
features[0][3] = 'Haralick'
features[0][4] = 'Label'


for file in glob.glob(path + '*.jpg'): 
    i = i+1
    print("File - ", i)
    head, tail = os.path.split(file)
    #print(head)
    #print(tail)
    # f_name has the file name with extension i.e. Y1.jpg, Y2.jpg
    f_name = tail
    features[j][0] = tail
    head2, tail2 = f_name.split(".")
    #print(head2)
    # filename has only the name i.e. Y1, Y2
    filename = head2
    #print(filename)
    
    
    # Read each image in im
    im = imageio.imread(file) 
    
    # Convert im from 3D to 2D
    image = color.rgb2gray(im)

    # Resize each image into height = 300 and width = 250
    img_resized = resize(image, (256, 256))
    
    # --------------- Entropy ---------------------
    entr = entropy(img_resized, disk(10))
    features[j][1] = np.mean(entr)
    
    # -------------- Local Binary Pattern--------------------------
    # settings for LBP
    radius = 3
    n_points = 8 * radius
    METHOD = 'uniform'
    lbp = local_binary_pattern(img_resized, n_points, radius, METHOD)
    features[j][2] = np.mean(lbp)
    
    # ------------------- Haralick Texture Feature Vector ---------------
    img = mt.imread(file) 
     
    # filtering the imagwe 
    img = img[:, :] 
      
    # setting gaussian filter 
    gaussian = mt.gaussian_filter(img, 15) 
   
    # setting threshold value 
    gaussian = (gaussian > gaussian.mean()) 
   
    # making is labeled image 
    labeled, n = mt.label(gaussian) 
    
    h_feature = mt.features.haralick(labeled) 
     
    #ht_mean = h_feature.mean(axis=0)
    features[j][3] = np.mean(h_feature)
    
    
    if (filename[0] == 'Y'):
        features[j][4] = 1   
    elif (filename[0] == 'N'):
        features[j][4] = 0
    
    
    j = j + 1


with open(path_out + 'Other_Features.csv', 'wt', newline ='') as file:
    writer1 = csv.writer(file, delimiter=',')
    for x in features:
        writer1.writerow(x)
file.close()